In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import keras
import random
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D, CuDNNLSTM, Flatten
from keras.layers import AveragePooling1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D,concatenate
from keras.layers import SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [0]:
import os
import warnings
from keras import backend as K
warnings.filterwarnings('ignore')

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [0]:
ls = '/content/gdrive/My Drive/Project MOF/'

In [0]:
with open(ls+'train_low_None_.pickle', 'rb') as f:
    Data = pickle.load(f)
Data.head()

,grid_0.0,grid_0.001,grid_0.002,grid_0.003,grid_0.004,grid_0.005,grid_0.006,grid_0.007,grid_0.008,grid_0.009000000000000001,...,grid_0.992,grid_0.993,grid_0.994,grid_0.995,grid_0.996,grid_0.997,grid_0.998,grid_0.999,grid_1.0,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009782,0.014963,0.011041,0.007322,0.009133,0.014201,0.016473,0.014228,0.010707,RASGAT
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009782,0.014963,0.011041,0.007322,0.009133,0.014201,0.016473,0.014228,0.010707,RASGAT
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009782,0.014963,0.011041,0.007322,0.009133,0.014201,0.016473,0.014228,0.010707,RASGAT
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009782,0.014963,0.011041,0.007322,0.009133,0.014201,0.016473,0.014228,0.010707,RASGAT
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009782,0.014963,0.011041,0.007322,0.009133,0.014201,0.016473,0.014228,0.010707,RASGAT


In [0]:
Data.shape

(148800, 1002)

In [0]:
# Train test split
idx = random.sample(range(148800), 29920) # 80% for training
all_index = list(np.setdiff1d(range(148800), idx))

# Getting X_train, X_val, y_train and scale them
X_train = Data.iloc[all_index,:].drop('label', axis = 1)
X_val = Data.iloc[idx,:].drop('label', axis = 1)
y_train = pd.get_dummies(Data.iloc[all_index,:]['label'])

names = y_train.columns
mapping = {}
i = 0
for n in names:
    mapping[i] = n
    i+=1

# scaler = StandardScaler()
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [0]:
# Construct CNN layers
# LeNet-5
cnn_model = Sequential()
cnn_model.add(Conv1D(6, 5, strides = 1, activation = 'relu', input_shape = (1001, 1)))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))
cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))
cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(120, activation = 'relu'))
cnn_model.add(Dense(84, activation = 'relu'))
cnn_model.add(Dense(186, activation = 'softmax', activity_regularizer = keras.regularizers.l2(0.1)))
cnn_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [0]:
# Fitting CNN model
cnn_model.fit(X_train.reshape(118880, 1001, 1), y_train.values, batch_size = 64, epochs = 100, verbose = 1)

# Validating the CNN modela
pred = cnn_model.predict(X_val.reshape(29920, 1001, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)

# Create category mapping dictionary
y_val = Data.iloc[idx,:]['label']
    
# Calculating accuracy
predicted = np.vectorize(mapping.get)(out)
print('Accuracy: '+ str(round(np.sum(predicted == y_val)/29920,2) * 100) + '%')

Epoch 1/100
118880/118880 [==============================] - 27s 229us/step - loss: 2.1418 - acc: 0.8579
Epoch 2/100
118880/118880 [==============================] - 27s 223us/step - loss: 1.9105 - acc: 0.9310
Epoch 3/100
118880/118880 [==============================] - 26s 222us/step - loss: 1.8900 - acc: 0.9374
Epoch 4/100
118880/118880 [==============================] - 27s 223us/step - loss: 1.8794 - acc: 0.9390
Epoch 5/100
118880/118880 [==============================] - 26s 223us/step - loss: 1.8717 - acc: 0.9417
Epoch 6/100
118880/118880 [==============================] - 27s 223us/step - loss: 1.8660 - acc: 0.9429
Epoch 7/100
118880/118880 [==============================] - 26s 222us/step - loss: 1.8615 - acc: 0.9443
Epoch 8/100
118880/118880 [==============================] - 27s 224us/step - loss: 1.8576 - acc: 0.9472
Epoch 9/100
118880/118880 [==============================] - 27s 224us/step - loss: 1.8545 - acc: 0.9489
Epoch 10/100
118880/118880 [===========================

In [0]:
# Save model
cnn_model.save("/content/gdrive/My Drive/Project MOF/cnn_v1.h5")

In [0]:
# Load and test on the experimental data
with open(ls+'test__None_.pickle', 'rb') as f:
    Test = pickle.load(f)

Test.head()

,grid_0.0,grid_0.001,grid_0.002,grid_0.003,grid_0.004,grid_0.005,grid_0.006,grid_0.007,grid_0.008,grid_0.009000000000000001,...,grid_0.992,grid_0.993,grid_0.994,grid_0.995,grid_0.996,grid_0.997,grid_0.998,grid_0.999,grid_1.0,label
0,0.033514,0.020624,0.027069,0.023330,0.000000,0.046661,0.039185,0.053880,0.034029,0.057747,...,0.034544,0.065352,0.048982,0.055169,0.043825,0.048208,0.050271,0.037638,0.054138,ZIF-67
1,0.034760,0.042744,0.025364,0.007046,0.096761,0.002820,0.030059,0.021604,0.025364,0.033349,...,0.568339,0.564113,0.563644,0.537812,0.566462,0.549084,0.538280,0.569748,0.620950,ZIF-67
2,0.032539,0.046020,0.042301,0.044392,0.039976,0.062521,0.064148,0.049273,0.067402,0.057640,...,0.038581,0.042765,0.022313,0.022429,0.041836,0.031144,0.028820,0.036722,0.043695,ZIF-67
3,0.029431,0.080294,0.040947,0.061420,0.055022,0.084453,0.068458,0.102047,0.063980,0.064299,...,0.066539,0.047345,0.050544,0.031030,0.039667,0.059501,0.039667,0.045106,0.047345,ZIF-67
4,0.038178,0.038881,0.037756,0.039795,0.033115,0.037334,0.038178,0.034873,0.033397,0.039303,...,0.013148,0.009843,0.012304,0.013007,0.015116,0.010617,0.010898,0.008156,0.011742,ZIF-8


In [0]:
Test.shape

(14, 1002)

In [0]:
X_test = scaler.transform(Test.iloc[:,:-1])
pred = cnn_model.predict(X_test.reshape(14, 1001, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)
predicted = np.vectorize(mapping.get)(out)
print(predicted)

['ZIF-65' 'AFIXES' 'ZIF-65' 'ZIF-65' 'ZIF-65' 'ZIF-65' 'ZIF-65' 'ZIF-65'
 'ZIF-12' 'ZIF-65' 'ZIF-90' 'ZIF-90' 'ZIF-12' 'ZIF-12']


In [0]:
preds = []
for p in pred:
  P = []
  for i in np.argsort(p)[::-1][0:3]:
    P.append(mapping[i])
  preds.append(P)

In [0]:
for p in preds:
  print(p)

['ZIF-65', 'ZIF-90', 'ZIF-8']
['AFIXES', 'GORQAF', 'DUT-23']
['ZIF-65', 'ZIF-90', 'TENQOS']
['ZIF-65', 'ZIF-90', 'ZIF-8']
['ZIF-65', 'ZIF-90', 'TENQOS']
['ZIF-65', 'ZIF-90', 'LITHUR']
['ZIF-65', 'ZIF-90', 'TENQOS']
['ZIF-65', 'ZIF-90', 'ZIF-8']
['ZIF-12', 'ZIF-71', 'ZIF-90']
['ZIF-65', 'ZIF-90', 'LITHUR']
['ZIF-90', 'ZIF-65', 'TENQOS']
['ZIF-90', 'ZIF-65', 'TENQOS']
['ZIF-12', 'ZIF-71', 'ZIF-90']
['ZIF-12', 'ZIF-71', 'ZIF-90']


In [0]:
Test.label

0     ZIF-67
1     ZIF-67
2     ZIF-67
3     ZIF-67
4      ZIF-8
5      ZIF-8
6      ZIF-8
7     ZIF-90
8     ZIF-71
9      ZIF-8
10    ZIF-90
11    ZIF-90
12    ZIF-71
13    ZIF-71
Name: label, dtype: object